# Export Model to TF Lite

In [1]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [0]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

In [0]:
NCLASSES = 2
HEIGHT = 224
WIDTH = 224
NUM_CHANNELS = 3
BATCH_SIZE = 128

In [4]:
base_model = tf.keras.applications.mobilenet.MobileNet(input_shape=(HEIGHT, WIDTH, NUM_CHANNELS), include_top=False, weights='imagenet')

17227776/17225924 [==============================] - 1s 0us/step


In [0]:
base_model.trainable = False

In [0]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(4096, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [0]:
import tensorflow_datasets as tfds

In [0]:
SPLIT_WEIGHTS = (8, 1, 1)
splits = tfds.Split.TRAIN.subsplit(weighted=SPLIT_WEIGHTS)

(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs', split=list(splits),
    with_info=True, as_supervised=True)

In [0]:
def format_imgs(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (HEIGHT, WIDTH))
  return image, label

In [0]:
train = raw_train.map(format_imgs)
validation = raw_validation.map(format_imgs)
test = raw_test.map(format_imgs)

In [0]:
SHUFFLE_BUFFER_SIZE = 1000

train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [0]:
import os
import datetime
%load_ext tensorboard.notebook

In [13]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

%tensorboard --logdir logdir

Reusing TensorBoard on port 6007 (pid 6529), started 1:31:40 ago. (Use '!kill 6529' to kill it.)

In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_batches, epochs=3, shuffle=True, validation_data=test_batches, callbacks=[tensorboard_callback])

Epoch 1/3
      1/Unknown - 7s 7s/step - loss: 0.7686 - accuracy: 0.4688

W0528 03:49:50.810078 139784376129408 callbacks.py:236] Method (on_train_batch_end) is slow compared to the batch update (0.219890). Check your callbacks.


582/582 [==============================] - 94s 161ms/step - loss: 0.1160 - accuracy: 0.9588 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/3
582/582 [==============================] - 87s 150ms/step - loss: 0.0653 - accuracy: 0.9742 - val_loss: 0.0616 - val_accuracy: 0.9754
Epoch 3/3
582/582 [==============================] - 87s 150ms/step - loss: 0.0513 - accuracy: 0.9809 - val_loss: 0.0497 - val_accuracy: 0.9802


## Get Concrete Function

In [0]:
conc_fn = tf.function(lambda x: model(x)).get_concrete_function(tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

## Convert to TF Lite

In [0]:
converter = tf.lite.TFLiteConverter.from_concrete_function(conc_fn)

In [17]:
tflite_model = converter.convert()

W0528 03:54:12.489787 139784376129408 meta_graph.py:447] Issue encountered when serializing global_step.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
to_proto not supported in EAGER mode.


KeyError: ignored

In [0]:
open('tflite_model.tflite', 'wb').write(tflite_model)

In [0]:
path = tf.keras.experimental.export_saved_model(model, './model')

In [0]:
!ls ./model

# Quantization

In [0]:
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

In [0]:
quantized_model = converter.convert()